In [1]:
import pyscheduling_cc.SMSP.riwiTi as smsp_riT
from pyscheduling_cc.SMSP.SingleMachine import Job

from docplex.cp.model import CpoModel
from docplex.cp.solver.cpo_callback import CpoCallback
from docplex.cp.expression import INTERVAL_MAX

In [2]:
def csp_transform_solution(msol, E_i, instance, objective):

    sol = instance.create_solution()
    k_tasks = []
    for i in range(instance.n):
        start = msol[E_i[i]][0]
        end = msol[E_i[i]][1]
        k_tasks.append(Job(i,start,end))

        k_tasks = sorted(k_tasks, key= lambda x: x[1])
        sol.machine.job_schedule = k_tasks

    if objective == "wiCi":
        sol.wiCi()
    elif objective == "wiTi":
        sol.objective_value = sol.machine.total_weighted_lateness(instance)
    return sol

In [3]:
instance = smsp_riT.riwiTi_Instance.generate_random(20,Wmax=6)
E = range(instance.n)

In [4]:
# Construct the model
model = CpoModel("smspModel")
objective = "wiTi"

# Jobs interval_vars including the release date and processing times constraints
E_i = []
for i in E:
    start_period = (instance.R[i], INTERVAL_MAX) if hasattr(instance, 'R') else (0, INTERVAL_MAX)
    job_i = model.interval_var( start = start_period,
                                size = instance.P[i], optional= False, name=f'E[{i}]')
    E_i.append(job_i)

# Sequential execution on the machine
machine_sequence = model.sequence_var( E_i, list(E) )
model.add( model.no_overlap(machine_sequence) )

# Define the objective 
if objective == "wiCi":
    model.add(model.minimize( sum( instance.W[i] * model.end_of(E_i[i]) for i in E ) )) # sum_{i in E} wi * ci
elif objective == "cmax":
    model.add(model.minimize( max( model.end_of(E_i[i]) for i in E ) )) # max_{i in E} ci 
elif objective == "wiTi":
    model.add( model.minimize( 
        sum( instance.W[i] * model.max(model.end_of(E_i[i]) - instance.D[i], 0) for i in E ) # sum_{i in E} wi * Ti
    ))

In [5]:
msol = model.solve(LogVerbosity="Normal", Workers=1, TimeLimit=30, LogPeriod=1000000,
                   log_output=True, trace_log=True, add_log_to_solution=True, RelativeOptimalityTolerance=0)

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Minimization problem - 21 variables, 1 constraint
 ! TimeLimit            = 30
 ! Workers              = 1
 ! LogPeriod            = 1000000
 ! RelativeOptimalityTolerance = 0
 ! Initial process time : 0.02s (0.02s extraction + 0.00s propagation)
 !  . Log search space  : 86.4 (before), 86.4 (after)
 !  . Memory usage      : 443.8 kB (before), 443.8 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0         21                 -
 + New bound is 0
 *           725       41  0.02s               (gap is 100.0%)
 *           554      115  0.02s               (gap is 100.0%)
 *           476      285  0.02s               (gap is 100.0%)
             476     2802          3        F        -
 + New bound is 61 (gap is 87.18%)
             476     2802     

In [6]:
sol = csp_transform_solution(msol, E_i, instance, objective)

In [7]:
print(sol)

Cmax : 348
Job_schedule (job_id , start_time , completion_time) | Completion_time
(2, 0, 1) : (16, 1, 4) : (18, 4, 6) : (1, 6, 13) : (12, 13, 19) : (3, 19, 21) : (5, 21, 28) : (6, 28, 31) : (17, 31, 38) : (8, 38, 42) : (9, 42, 49) : (13, 49, 58) : (10, 58, 62) : (0, 62, 70) : (7, 70, 75) : (15, 75, 81) : (11, 81, 89) : (19, 89, 97) : (14, 97, 106) : (4, 106, 115) | 348


In [11]:
solve_result = smsp_riT.Heuristics.ACT_WSECi(instance)
print(solve_result)

Search stopped with status : FEASIBLE
 Solution is : 
 Cmax : 437
Job_schedule (job_id , start_time , completion_time) | Completion_time
(18, 3, 5) : (16, 5, 8) : (12, 8, 14) : (6, 14, 17) : (3, 17, 19) : (2, 19, 20) : (1, 20, 27) : (5, 27, 34) : (9, 34, 41) : (8, 41, 45) : (13, 45, 54) : (17, 54, 61) : (10, 61, 65) : (0, 65, 73) : (7, 73, 78) : (15, 78, 84) : (11, 84, 92) : (19, 92, 100) : (14, 100, 109) : (4, 109, 118) | 437 
Runtime is : 0.00034176098415628076s 
time to best is : -1s 

